In [2]:
import pandas as pd
import numpy as np
import json
import os
import re
from great_expectations.core import ExpectationSuite
from great_expectations.dataset import PandasDataset
from great_expectations.render.renderer import ValidationResultsPageRenderer
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import DataContextConfig

# Let's create a sample CSV for demonstration
def create_sample_data():
    """Create a sample dataset with Name, Email, Age columns"""
    data = {
        'Name': ['John Doe', 'Jane Smith', None, 'Robert Brown', 'Emily White', 'David Green'],
        'Email': ['john@example.com', 'jane.smith@company.org', 'invalid-email', None, 'emily@domain.com', 'emily@domain.com'],
        'Age': [34, 28, 45, None, 22, 'thirty']
    }
    df = pd.DataFrame(data)
    df.to_csv('sample_data.csv', index=False)
    return df

# Task 1: Understanding and Defining Data Quality Metrics
def calculate_basic_metrics(df):
    """Calculate basic data quality metrics: completeness, validity, uniqueness"""
    
    # Calculate completeness (percentage of non-null values)
    completeness = {}
    for column in df.columns:
        non_null_count = df[column].count()
        total_count = len(df)
        completeness[column] = (non_null_count / total_count) * 100
    
    # Calculate validity (% of email fields containing @)
    # Only applicable for the Email column
    valid_emails = sum(1 for email in df['Email'] if isinstance(email, str) and '@' in email)
    email_validity = (valid_emails / len(df)) * 100
    
    # Calculate uniqueness (count distinct entries in Email column)
    unique_emails = df['Email'].nunique()
    email_uniqueness = (unique_emails / len(df)) * 100
    
    # Create a metrics dictionary
    metrics = {
        'completeness': completeness,
        'email_validity': email_validity,
        'email_uniqueness': email_uniqueness
    }
    
    return metrics

# Task 2: Calculating Data Quality Score
def calculate_quality_score(metrics):
    """Aggregate multiple metrics to calculate an overall data quality score"""
    
    # Calculate average completeness across all columns
    avg_completeness = sum(metrics['completeness'].values()) / len(metrics['completeness'])
    
    # Use all three metrics for the overall score
    overall_score = (avg_completeness + metrics['email_validity'] + metrics['email_uniqueness']) / 3
    
    return overall_score

# Task 3: Creating Expectations for a CSV
def create_expectations():
    """Develop basic data quality expectations using Great Expectations"""
    
    # Create an ExpectationSuite
    suite = ExpectationSuite(expectation_suite_name="sample_data_suite")
    
    # Load your data as a PandasDataset
    df = pd.read_csv('sample_data.csv')
    dataset = PandasDataset(df, expectation_suite=suite)
    
    # Define expectations for completeness
    dataset.expect_column_values_to_not_be_null('Name')
    dataset.expect_column_values_to_not_be_null('Email')
    dataset.expect_column_values_to_not_be_null('Age')
    
    # Define expectations for validity
    dataset.expect_column_values_to_match_regex('Email', r'.+@.+\..+')
    dataset.expect_column_values_to_be_of_type('Age', 'int64', mostly=0.9)
    
    # Define expectations for uniqueness
    dataset.expect_column_values_to_be_unique('Email')
    
    # Save the expectation suite to a JSON file
    with open('expectation_suite.json', 'w') as f:
        json.dump(dataset.get_expectation_suite().to_json_dict(), f, indent=2)
    
    return dataset.get_expectation_suite()

# Task 4: Running and Validating Expectations
def validate_data(df, expectation_suite):
    """Run the created expectations and generate an output report"""
    
    # Convert the DataFrame to a PandasDataset with the expectation suite
    dataset = PandasDataset(df, expectation_suite=expectation_suite)
    
    # Validate the data against the expectations
    validation_result = dataset.validate()
    
    # Print a summary of the validation results
    print(f"Validation successful: {validation_result.success}")
    print(f"Total expectations: {len(validation_result.results)}")
    print(f"Passed expectations: {sum(1 for result in validation_result.results if result.success)}")
    
    # Create a simple HTML report
    with open('validation_report.html', 'w') as f:
        f.write("<html><body>")
        f.write("<h1>Data Validation Report</h1>")
        f.write(f"<p>Validation successful: {validation_result.success}</p>")
        f.write("<h2>Results</h2>")
        f.write("<table border='1'>")
        f.write("<tr><th>Expectation</th><th>Success</th><th>Details</th></tr>")
        
        for result in validation_result.results:
            expectation = result.expectation_config
            f.write("<tr>")
            f.write(f"<td>{expectation.expectation_type}</td>")
            f.write(f"<td>{'✅' if result.success else '❌'}</td>")
            
            # Add some details about the expectation
            details = f"Column: {expectation.kwargs.get('column', 'N/A')}"
            if 'mostly' in expectation.kwargs:
                details += f", Threshold: {expectation.kwargs['mostly'] * 100}%"
            f.write(f"<td>{details}</td>")
            
            f.write("</tr>")
        
        f.write("</table>")
        f.write("</body></html>")
    
    return validation_result

# Task 5: Automating Data Quality Score Calculation
def automate_quality_score(file_path):
    """Automate the data quality score calculation"""
    
    # Load the data
    df = pd.read_csv(file_path)
    
    # Calculate basic metrics
    metrics = calculate_basic_metrics(df)
    
    # Calculate the quality score
    score = calculate_quality_score(metrics)
    
    # Load the expectation suite
    with open('expectation_suite.json', 'r') as f:
        expectation_suite_dict = json.load(f)
    
    # Convert dict to ExpectationSuite
    expectation_suite = ExpectationSuite(
        expectation_suite_name=expectation_suite_dict["expectation_suite_name"],
        expectations=expectation_suite_dict["expectations"]
    )
    
    # Validate the data against the expectations
    validation_result = validate_data(df, expectation_suite)
    
    # Return the quality score and validation result
    return score, validation_result

# Task 6: Leveraging Data Quality Metrics for Automated Data Cleaning
def clean_data(df, validation_result):
    """Clean data based on validation results"""
    
    cleaned_df = df.copy()
    
    # Track the number of rows and values cleaned
    cleaning_stats = {
        'rows_affected': 0,
        'cells_cleaned': 0,
        'cleaning_actions': []
    }
    
    # Identify failed expectations
    failed_expectations = [result.expectation_config for result in validation_result.results if not result.success]
    
    for expectation in failed_expectations:
        expectation_type = expectation.expectation_type
        column = expectation.kwargs.get('column')
        
        # Clean null values
        if expectation_type == 'expect_column_values_to_not_be_null' and column:
            # For demo purposes, we'll use simple imputation strategies
            if column == 'Name':
                # Fill missing names with placeholder
                missing_mask = cleaned_df[column].isna()
                cleaned_df.loc[missing_mask, column] = 'Unknown User'
                cleaning_stats['cells_cleaned'] += missing_mask.sum()
                cleaning_stats['rows_affected'] += missing_mask.sum()
                cleaning_stats['cleaning_actions'].append(f"Filled {missing_mask.sum()} missing values in {column}")
            
            elif column == 'Email':
                # Generate placeholder emails based on name
                missing_mask = cleaned_df[column].isna()
                for idx in cleaned_df[missing_mask].index:
                    name = cleaned_df.loc[idx, 'Name']
                    if pd.isna(name) or name == 'Unknown User':
                        email = f"user{idx}@placeholder.com"
                    else:
                        # Create an email from the name
                        name_parts = name.lower().split()
                        email = f"{name_parts[0]}.{name_parts[-1]}@placeholder.com"
                    
                    cleaned_df.loc[idx, column] = email
                    cleaning_stats['cells_cleaned'] += 1
                    cleaning_stats['rows_affected'] += 1
                
                cleaning_stats['cleaning_actions'].append(f"Generated {missing_mask.sum()} missing emails")
            
            elif column == 'Age':
                # Fill missing ages with the median age
                median_age = cleaned_df[column].median()
                if pd.isna(median_age):  # If all values are null
                    median_age = 30  # Default value
                
                missing_mask = cleaned_df[column].isna()
                cleaned_df.loc[missing_mask, column] = median_age
                cleaning_stats['cells_cleaned'] += missing_mask.sum()
                cleaning_stats['rows_affected'] += missing_mask.sum()
                cleaning_stats['cleaning_actions'].append(f"Filled {missing_mask.sum()} missing ages with median: {median_age}")
        
        # Fix invalid emails
        if expectation_type == 'expect_column_values_to_match_regex' and column == 'Email':
            # Find emails that don't match the regex pattern
            invalid_mask = ~cleaned_df[column].str.contains('@', na=False)
            for idx in cleaned_df[invalid_mask].index:
                name = cleaned_df.loc[idx, 'Name']
                if pd.isna(name) or name == 'Unknown User':
                    email = f"user{idx}@corrected.com"
                else:
                    # Create an email from the name
                    name_parts = name.lower().split()
                    email = f"{name_parts[0]}.{name_parts[-1]}@corrected.com"
                
                cleaned_df.loc[idx, column] = email
                cleaning_stats['cells_cleaned'] += 1
                cleaning_stats['rows_affected'] += 1
            
            cleaning_stats['cleaning_actions'].append(f"Corrected {invalid_mask.sum()} invalid email formats")
        
        # Fix data types
        if expectation_type == 'expect_column_values_to_be_of_type' and column == 'Age':
            # Convert non-numeric ages to numeric
            for idx in cleaned_df.index:
                age_value = cleaned_df.loc[idx, column]
                if not pd.isna(age_value) and not isinstance(age_value, (int, float)):
                    try:
                        # Try to convert string numbers to int
                        cleaned_df.loc[idx, column] = int(age_value)
                    except ValueError:
                        # For text like "thirty", use a default value
                        cleaned_df.loc[idx, column] = 30
                        cleaning_stats['cells_cleaned'] += 1
                        cleaning_stats['rows_affected'] += 1
            
            cleaning_stats['cleaning_actions'].append("Converted non-numeric ages to numeric values")
        
        # Fix duplicate emails
        if expectation_type == 'expect_column_values_to_be_unique' and column == 'Email':
            # Find duplicate emails
            duplicates = cleaned_df[cleaned_df.duplicated(subset=[column], keep='first')][column].tolist()
            
            if duplicates:
                # Make duplicated emails unique by adding a counter
                counter = {}
                for idx in cleaned_df.index:
                    email = cleaned_df.loc[idx, column]
                    
                    if email in duplicates:
                        if email not in counter:
                            counter[email] = 1
                        else:
                            counter[email] += 1
                            # Modify the duplicated email to make it unique
                            base_email = email.split('@')
                            new_email = f"{base_email[0]}+{counter[email]}@{base_email[1]}"
                            cleaned_df.loc[idx, column] = new_email
                            cleaning_stats['cells_cleaned'] += 1
                            cleaning_stats['rows_affected'] += 1
                
                cleaning_stats['cleaning_actions'].append(f"Made {sum(counter.values()) - len(counter)} duplicate emails unique")
    
    return cleaned_df, cleaning_stats

# Main function to execute the full data quality workflow
def main():
    """Execute the full data quality workflow"""
    
    # Create sample data
    print("Creating sample data...")
    df = create_sample_data()
    print(df)
    print("\n" + "-"*50 + "\n")
    
    # Task 1: Calculate basic metrics
    print("Calculating basic data quality metrics...")
    metrics = calculate_basic_metrics(df)
    print(f"Completeness: {metrics['completeness']}")
    print(f"Email Validity: {metrics['email_validity']:.2f}%")
    print(f"Email Uniqueness: {metrics['email_uniqueness']:.2f}%")
    print("\n" + "-"*50 + "\n")
    
    # Task 2: Calculate quality score
    print("Calculating overall data quality score...")
    score = calculate_quality_score(metrics)
    print(f"Overall Data Quality Score: {score:.2f}%")
    print("\n" + "-"*50 + "\n")
    
    # Task 3: Create expectations
    print("Creating expectations using Great Expectations...")
    expectation_suite = create_expectations()
    print("Expectations created and saved to 'expectation_suite.json'")
    print("\n" + "-"*50 + "\n")
    
    # Task 4: Validate data
    print("Validating data against expectations...")
    validation_result = validate_data(df, expectation_suite)
    print("Validation report saved to 'validation_report.html'")
    print("\n" + "-"*50 + "\n")
    
    # Task 5 & 6: Automated cleaning based on quality metrics
    print("Checking if data cleaning is needed...")
    # Define a quality threshold
    QUALITY_THRESHOLD = 80.0
    
    if score < QUALITY_THRESHOLD:
        print(f"Data quality score ({score:.2f}%) is below threshold ({QUALITY_THRESHOLD}%).")
        print("Initiating automated data cleaning...")
        
        # Clean the data
        cleaned_df, cleaning_stats = clean_data(df, validation_result)
        
        # Save the cleaned data
        cleaned_df.to_csv('cleaned_data.csv', index=False)
        
        # Print cleaning statistics
        print("\nCleaning Statistics:")
        print(f"Rows affected: {cleaning_stats['rows_affected']}")
        print(f"Cells cleaned: {cleaning_stats['cells_cleaned']}")
        print("\nCleaning actions performed:")
        for action in cleaning_stats['cleaning_actions']:
            print(f"- {action}")
        
        # Recalculate metrics and score for the cleaned data
        new_metrics = calculate_basic_metrics(cleaned_df)
        new_score = calculate_quality_score(new_metrics)
        
        print("\nNew Data Quality Score after cleaning: {:.2f}%".format(new_score))
        print("\nCleaned data saved to 'cleaned_data.csv'")
        
        # Display the cleaned data
        print("\nCleaned Data:")
        print(cleaned_df)
    else:
        print(f"Data quality score ({score:.2f}%) is above threshold ({QUALITY_THRESHOLD}%).")
        print("No cleaning needed.")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'great_expectations.dataset'